In [ ]:
import params, prepare
from model_loader import ModelLoader
from initial_freezing import InitialFreezing
from selective_freezing import SelectiveFreezing
from model_trainer import ModelTrainer
from transformers import TrainingArguments

In [ ]:
model_name = 'skt/kogpt2-base-v2'  # 사용할 모델 이름
data_path = './data/rawdata.csv'  # 데이터 파일 경로

In [ ]:
# 모델 및 토크나이저 로드
model_loader = ModelLoader(model_name)
model, tokenizer = model_loader.load()

In [ ]:
df = prepare.load_data(data_path)

In [ ]:
train_df, test_df = prepare.split_data(df, test_size=0.2)  # 데이터를 학습 및 평가 데이터로 분리

In [ ]:
# 데이터 로드 및 준비
train_dataset = prepare.prepare_dataset(train_df, tokenizer)  # 학습 데이터셋 준비
test_dataset = prepare.prepare_dataset(test_df, tokenizer)  # 평가 데이터셋 준비

In [ ]:
# 초기 프리징 및 점진적 언프리징
initial_freezing = InitialFreezing(model, num_unfreeze_layers=2)  # 초기 프리징 적용

In [ ]:
# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir=params.OUTPUT_DIR,  # 모델 출력 디렉토리
    num_train_epochs=params.NUM_TRAIN_EPOCHS,  # 학습 에포크 수
    per_device_train_batch_size=params.BATCH_SIZE,  # 학습 배치 크기
    per_device_eval_batch_size=params.BATCH_SIZE,  # 평가 배치 크기
    learning_rate=params.LEARNING_RATE,  # 학습률
    logging_dir=params.LOGGING_DIR,  # 로깅 디렉토리
    logging_steps=params.LOGGING_STEPS,  # 로깅 빈도
    save_steps=params.SAVE_STEPS,  # 모델 저장 빈도
    evaluation_strategy=params.EVALUATION_STRATEGY,  # 평가 전략
    eval_steps=params.EVAL_STEPS,  # 평가 빈도
    save_total_limit=params.SAVE_TOTAL_LIMIT,  # 저장할 체크포인트의 최대 수
)

In [ ]:
# 모델 학습 및 검증
trainer = ModelTrainer(model, tokenizer)
trainer.set_training_args()  # 학습 인자 설정

In [ ]:
# 모델 학습
trainer.train(train_dataset, test_dataset)
trainer.save_model(params.OUTPUT_DIR)

In [ ]:
# 시각화
trainer.plot_metrics()

# initial_freezing

In [ ]:
import pandas as pd
a = pd.read_csv('data/rawdata.csv', index_col=0)
b = a[:3000]
b.to_csv('data/test_raw_data2.csv')

In [1]:
import params  # 파라미터 모듈 임포트
import prepare  # 데이터 준비 모듈 임포트
from transformers import TrainingArguments
from model_loader import ModelLoader
from initial_freezing import InitialFreezing
from model_trainer import ModelTrainer
import torch

2024-06-03 18:03:39.965183: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-03 18:03:40.027444: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 18:03:40.614702: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# GPU 가용성 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available. Using CPU.")

GPU is available. Using GPU: NVIDIA GeForce GTX 1050 Ti


In [3]:
model_name = 'skt/kogpt2-base-v2'  # 사용할 모델 이름
model_loader = ModelLoader(model_name)
model, tokenizer = model_loader.load()
model.to(device)  # 모델을 GPU로 이동

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51206, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51206, bias=False)
)

In [4]:
data_path = './data/test_raw_data.csv'  # 데이터 파일 경로
df = prepare.load_data(data_path)
train_df, test_df = prepare.split_data(df, test_size=0.2)  # 데이터를 학습 및 평가 데이터로 분리
train_dataset = prepare.prepare_dataset(train_df, tokenizer)  # 학습 데이터셋 준비
test_dataset = prepare.prepare_dataset(test_df, tokenizer)  # 평가 데이터셋 준비

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
# 초기 프리징
initial_freezing = InitialFreezing(model, num_unfreeze_layers=2)
initial_freezing.apply()

In [6]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir=params.OUTPUT_DIR,  # 모델 출력 디렉토리
    num_train_epochs=params.NUM_TRAIN_EPOCHS,  # 학습 에포크 수
    per_device_train_batch_size=params.BATCH_SIZE,  # 학습 배치 크기
    per_device_eval_batch_size=params.BATCH_SIZE,  # 평가 배치 크기
    learning_rate=params.LEARNING_RATE,  # 학습률
    logging_dir=params.LOGGING_DIR,  # 로깅 디렉토리
    logging_steps=params.LOGGING_STEPS,  # 로깅 빈도
    save_steps=params.SAVE_STEPS,  # 모델 저장 빈도
    evaluation_strategy=params.EVALUATION_STRATEGY,  # 평가bb 전략
    eval_steps=params.EVAL_STEPS,  # 평가 빈도
    save_total_limit=params.SAVE_TOTAL_LIMIT,  # 저장할 체크포인트의 최대 수
    load_best_model_at_end=True,  # EarlyStoppingCallback를 위한 설정
    fp16=True,  # 혼합 정밀도 학습(Mixed Precision Training) 활성화 (필요 시)
    gradient_accumulation_steps=16  # 기울기 누적(Gradient Accumulation) 활성화 (필요 시)
)

/home/bok/.local/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
trainer = ModelTrainer(model, tokenizer)
trainer.set_training_args(training_args)  # 학습 인자 설정

In [8]:
# 모델 학습
trainer.train(train_dataset, test_dataset)
trainer.save_model(params.OUTPUT_DIR)

Step,Training Loss,Validation Loss


In [ ]:
# 시각화
output_file = './gpt2_visualization/gpt2_model_v1_metrics.png'  # 저장할 파일 경로
trainer.plot_metrics(output_file)

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

model_path = './results'  # 모델 및 토크나이저가 저장된 디렉터리
num_beams = 5  # 빔 너비: 가능성이 높은 5개의 후보를 유지하고 확장
max_length = 50  # 생성할 텍스트의 최대 길이
num_return_sequences = 1  # 생성할 텍스트의 수
no_repeat_ngram_size = 2  # n-그램 반복 금지 설정: 두 단어 조합 반복 금지

tokenizer = GPT2TokenizerFast.from_pretrained(model_path)  # 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained(model_path)  # 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU가 가용한 경우 GPU 사용
model.to(device)  # 모델을 GPU로 이동
model.eval()  # 평가 모드로 전환